# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [1]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

2025-10-28 09:55:11,289 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [2]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are an experienced Database Administrator (DBA) tasked with designing a normalized SQL database schema for the Momentum Onboarding Platform based on the Product Requirements Document (PRD) provided below.

**Your Task:**
1. Analyze the PRD to identify all the core entities and their relationships
2. Design a normalized SQL schema (at least 3NF) with a minimum of two tables
3. Include appropriate primary keys, foreign keys, and constraints
4. Add indexes where appropriate for performance
5. Consider the user personas (New Hires, Managers, HR Admins) and their different needs
6. Output ONLY the raw CREATE TABLE statements - no explanations, comments, or markdown formatting

**Key Entities to Consider (at minimum):**
- Users (with different roles: new hire, manager, HR admin, buddy)
- Onboarding tasks and templates
- Task assignments and their completion status
- Any other entities you identify as necessary from the PRD

**Requirements:**
- Use SQLite-compatible syntax
- Include appropriate data types (TEXT, INTEGER, REAL, BLOB, or DATE/DATETIME)
- Add NOT NULL constraints where data is required
- Use foreign keys to establish relationships between tables
- Consider timestamps for tracking creation and updates
- Ensure the schema supports all the user stories in the PRD

**PRD Content:**
{prd_content}

Generate the SQL CREATE TABLE statements now:
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    full_name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    sso_identifier TEXT UNIQUE,
    role TEXT NOT NULL CHECK(role IN ('new_hire', 'manager', 'hr_admin')),
    manager_id INTEGER,
    hire_date DATE,
    created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (manager_id) REFERENCES users(user_id) ON DELETE SET NULL
);

CREATE TABLE templates (
    template_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE,
    description TEXT,
    template_type TEXT NOT NULL CHECK(template_type IN ('ONBOARDING_JOURNEY', 'ROLE_PLAYBOOK', '30_60_90_PLAN')),
    created_by_user_id INTEGER NOT NULL,
    created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (created_by_user_id) REFERENCES users(user_id)
);

CREATE TABLE template_tasks (
    template_task_i

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [3]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
You are an experienced Database Administrator (DBA) tasked with generating realistic seed data for the Momentum Onboarding Platform database.

**Your Task:**
Generate 5-10 realistic INSERT statements for each table in the schema provided below. The data should be:
1. Contextually appropriate for a new hire onboarding system
2. Consistent across related tables (foreign keys must reference existing primary keys)
3. Realistic and representative of actual onboarding scenarios
4. Diverse enough to demonstrate the system's capabilities

**Data Guidelines:**
- **Users**: Include a mix of roles (HR admin, hiring managers, new hires, buddies)
  - Example names: Sarah Chen (HR Admin), David Martinez (Manager), Amelia Johnson (New Hire), etc.
  - Use realistic email addresses following a corporate pattern (e.g., firstname.lastname@momentum.com)
  
- **Onboarding Tasks**: Include realistic tasks such as:
  - "Complete HR Paperwork" (pre-boarding)
  - "Set up laptop and accounts" (IT setup)
  - "Review company handbook" (compliance)
  - "Schedule 1-on-1 with manager" (orientation)
  - "Complete Security Training" (compliance/mandatory)
  - "Meet your onboarding buddy" (team integration)
  - "First quick win project: Update team wiki" (first project)
  
- **Task Status**: Mix of pending, in_progress, and completed tasks
- **Dates**: Use realistic timestamps (recent dates, logical sequences)
- **Templates**: Include at least one generic onboarding template and one role-specific template (e.g., Engineering or Sales)

**Output Requirements:**
- Generate ONLY raw SQL INSERT statements
- No explanations, comments, or markdown formatting
- Ensure data integrity (all foreign keys reference valid primary keys)
- Use proper SQLite syntax for dates and text escaping
- Insert data in the correct order (parent tables before child tables that reference them)

**Database Schema:**
{cleaned_schema}

**PRD Context (for understanding the domain):**
{prd_content}

Generate the SQL INSERT statements now:
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
INSERT INTO users (user_id, full_name, email, sso_identifier, role, manager_id, hire_date, created_at, updated_at) VALUES
(1, 'Sarah Chen', 'sarah.chen@momentum.com', 'auth0|sarah.chen', 'hr_admin', NULL, '2022-01-15', '2022-01-10 09:00:00', '2022-01-10 09:00:00'),
(2, 'David Martinez', 'david.martinez@momentum.com', 'auth0|david.martinez', 'manager', NULL, '2022-03-20', '2022-03-15 11:30:00', '2022-03-15 11:30:00'),
(3, 'Emily White', 'emily.white@momentum.com', 'auth0|emily.white', 'manager', NULL, '2022-05-11', '2022-05-01 14:00:00', '2022-05-01 14:00:00'),
(4, 'Amelia Johnson', 'amelia.johnson@momentum.com', 'auth0|amelia.johnson', 'new_hire', 2, '2023-10-23', '2023-10-01 10:00:00', '2023-10-01 10:00:00'),
(5, 'Ben Carter', 'ben.carter@momentum.com', 'auth0|ben.carter', 'new_hire', 3, '2023-10-30', '2023-10-15 16:20:00', '2023-10-15 16:20:00'),
(6, 'Chloe Davis', 'chloe.davis@momentum.com', 'auth0|chloe.davis', 'new_hire', 2, '2023-11-06', '2023-10-25 0

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [4]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # TODO: Read the content of the schema file using load_artifact.
        schema_sql = load_artifact(schema_path)
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        if schema_sql:
            cursor.executescript(schema_sql)
            print("Tables created successfully.")
        else:
            print("Error: Schema file is empty or could not be read.")
            return

        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            # Load and execute the seed data
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")
            else:
                print("Warning: Seed data file is empty or could not be read.")
        else:
            print(f"Warning: Seed data file not found at {seed_path}. Skipping seed data insertion.")

        # TODO: Commit the changes to the database.
        conn.commit()
        print("Database changes committed successfully.")
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Successfully connected to database at /Users/brianfisher/trainingRepos/AG-AISOFTDEV/artifacts/onboarding.db
Tables created successfully.
Seed data inserted successfully.
Database changes committed successfully.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.